In [4]:
import cv2
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder

def preprocess_image(image_path, target_size=(224, 224)):
    # Load image
    image = cv2.imread(image_path)
    # Resize image
    image = cv2.resize(image, target_size)
    # Normalize pixel values to [0, 1]
    image = image / 255.0
    return image

# Example: Preprocess all images in a folder
def preprocess_dataset(data_dir, target_size=(224, 224)):
    images = []
    labels = []
    class_names = os.listdir(data_dir)
    for class_name in class_names:
        class_dir = os.path.join(data_dir, class_name)
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            image = preprocess_image(image_path, target_size)
            images.append(image)
            labels.append(class_name)
    return np.array(images), np.array(labels)
# Preprocess the dataset
data_dir = "Data"
images, labels = preprocess_dataset(data_dir)

In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Convert string labels to numerical labels
label_encoder = LabelEncoder()
integer_labels = label_encoder.fit_transform(labels)

# Convert numerical labels to one-hot encoding
onehot_encoder = OneHotEncoder(sparse_output=False)
integer_labels = integer_labels.reshape(len(integer_labels), 1)
onehot_labels = onehot_encoder.fit_transform(integer_labels)

In [10]:
from sklearn.model_selection import train_test_split

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(images, onehot_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

c:\Users\Arsh Bhushan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,347 (42.61 MB)

 Trainable params: 11,169,347 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 18s 579ms/step - accuracy: 0.6615 - loss: 1.0866 - val_accuracy: 1.0000 - val_loss: 0.0124
Epoch 2/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 16s 574ms/step - accuracy: 0.9973 - loss: 0.0070 - val_accuracy: 1.0000 - val_loss: 5.5331e-05
Epoch 3/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 14s 517ms/step - accuracy: 1.0000 - loss: 2.6265e-05 - val_accuracy: 1.0000 - val_loss: 5.2197e-06
Epoch 4/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 14s 510ms/step - accuracy: 1.0000 - loss: 3.8394e-06 - val_accuracy: 1.0000 - val_loss: 1.7819e-06
Epoch 5/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 14s 511ms/step - accuracy: 1.0000 - loss: 1.1891e-06 - val_accuracy: 1.0000 - val_loss: 1.1491e-06
Epoch 6/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 14s 516ms/step - accuracy: 1.0000 - loss: 1.2454e-06 - val_accuracy: 1.0000 - val_loss: 1.0943e-06
Epoch 7/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 16s 613ms/step - accuracy: 1.0000 - loss: 9.8725e-07 - val_accuracy: 1.0000 - val_loss: 9.2428e-07
Epoch 8/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 547ms/step 

In [13]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 108ms/step - accuracy: 1.0000 - loss: 1.2657e-06
Test Accuracy: 100.00%


In [14]:
# Save the model
model.save("sign_language_model.h5")

In [18]:
# Load the model
model = tf.keras.models.load_model("sign_language_model.h5")

# Preprocess a new image
new_image = preprocess_image("Data/C/single_24.jpg")

# Make a prediction
predictions = model.predict(np.array([new_image]))
predicted_class = label_encoder.inverse_transform([np.argmax(predictions)])[0]
print(f"Predicted Class: {predicted_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Predicted Class: C
